In [1]:
!pip install -qU ipython
%load_ext autoreload
%autoreload 2

In [2]:
cd titanic

/kaggle/working/titanic


In [3]:
import pandas as pd

path = "/kaggle/input/competitions/titanic/train.csv"
df = pd.read_csv(path)

In [4]:
from utils import fillna_with_mean
